# 6장 순수함수적상태

목표 : 난수발생기 예제로 통해서  순수함수적 프로그램을 작성하는 방법을 배워보자.

## 6.1 부수효과를 이용한 난수 발생

In [1]:
val rng = new scala.util.Random

rng: util.Random = scala.util.Random@57ada944

In [2]:
rng.nextDouble

res1: Double = 0.8764510441268563

In [3]:
rng.nextDouble

res2: Double = 0.7854612353603011

In [4]:
rng.nextInt

res3: Int = -414294773

In [5]:
rng.nextInt( 10 )

res4: Int = 1

## 6.2 순수함수적 난수 발생

- 순수함수를 방법 
상태를 부수효과로서 갱신하지 말고,  난수와 새로운 상태를 갖는 새로운 난수생성기를 같이 반환

In [65]:
trait RNG {
    def nextInt : (Int, RNG)
}

defined trait RNG

In [66]:
case class SimpleRNG( seed: Long) extends RNG {
    def nextInt : ( Int, RNG ) = {
        val newSeed = ( seed * 0x5DEECE66DL + 0xBL )  & 0xFFFFFFFFFFFFL
        val nextRNG = SimpleRNG(newSeed)
        val n = ( newSeed >>> 16 ).toInt
        ( n, nextRNG)
    }
}

defined class SimpleRNG

In [9]:
val rng = SimpleRNG( 42 )

rng: SimpleRNG = SimpleRNG(42L)

In [10]:
val ( n1, rng2 ) = rng.nextInt

n1: Int = 16159453
rng2: RNG = SimpleRNG(1059025964525)

In [11]:
val ( n2, rng3 ) = rng.nextInt

n2: Int = 16159453
rng3: RNG = SimpleRNG(1059025964525)

### 연습문제 6.1
RNG.nextInt를 이용해서 0 이상, Int.MaxVale 이하 난수를 생성하는 함수 작성

In [67]:
def nonNegativeInt(rng: RNG) : (Int, RNG) = {
   val (n , newRng )  = rng.nextInt
   if( n < 0 )  ( -n, newRng )
   else ( n, newRng )
}

val myRNG = SimpleRNG( 42 )
val ( nonNeg1 , myRNG1 ) = nonNegativeInt( myRNG )
val ( nonNeg2 , myRNG2 ) = nonNegativeInt( myRNG1 )
val ( nonNeg3 , myRNG3 ) = nonNegativeInt( myRNG2 )
val ( nonNeg4 , myRNG4 ) = nonNegativeInt( myRNG3 )
val ( nonNeg5 , myRNG5 ) = nonNegativeInt( myRNG4 )

defined function nonNegativeInt
myRNG: SimpleRNG = SimpleRNG(42L)
nonNeg1: Int = 16159453
myRNG1: RNG = SimpleRNG(1059025964525)
nonNeg2: Int = 1281479697
myRNG2: RNG = SimpleRNG(197491923327988)
nonNeg3: Int = 340305902
myRNG3: RNG = SimpleRNG(259172689157871)
nonNeg4: Int = 2015756020
myRNG4: RNG = SimpleRNG(149370390209998)
nonNeg5: Int = 1770001318
myRNG5: RNG = SimpleRNG(115998806404289)

### 연습문제 6.2
0이상, 1미만의 Double난수를 발생하는 함수 작성

In [68]:
def double(rng : RNG ) : (Double, RNG) = {
    val (n , newRng )  = nonNegativeInt( rng )
    ( n / Double.MaxValue + 1 , newRng )
}
val myRNG = SimpleRNG( 11 )
val ( nonNeg1 , myRNG1 ) = nonNegativeInt( myRNG )
val ( nonNeg2 , myRNG2 ) = nonNegativeInt( myRNG1 )
val ( nonNeg3 , myRNG3 ) = nonNegativeInt( myRNG2 )
val ( nonNeg4 , myRNG4 ) = nonNegativeInt( myRNG3 )
val ( nonNeg5 , myRNG5 ) = nonNegativeInt( myRNG4 )

defined function double
myRNG: SimpleRNG = SimpleRNG(11L)
nonNeg1: Int = 4232237
myRNG1: RNG = SimpleRNG(277363943098)
nonNeg2: Int = 178803790
myRNG2: RNG = SimpleRNG(11718085204285)
nonNeg3: Int = 758674372
myRNG3: RNG = SimpleRNG(49720483695876)
nonNeg4: Int = 1565954732
myRNG4: RNG = SimpleRNG(102626409374399)
nonNeg5: Int = 392261992
myRNG5: RNG = SimpleRNG(25707281917278)

### 연습문제 6.3
(Int, Double), ( Double, Int ), (Double, Double, Double )을 발생하는 함수 작성
앞에서 작성한 함수들을 재사용

In [57]:
def intDouble(rng : RNG ) : ((Int, Double), RNG ) = {
    val first = nonNegativeInt( rng )
    val second = double( first._2 )
    
    ((first._1, second._1 ), second._2 )
}

def DoubleInt(rng : RNG ) : ((Double, Int), RNG ) = {
    
    val first = double( rng )
    val second = nonNegativeInt( first._2 )

    ((first._1, second._1 ), second._2 )
}

def double3(rng : RNG ) : ((Double, Double, Double), RNG )  = {
    val first = double( rng )
    val second = double( first._2 )
    val third = double( second._2 )
    ((first._1, second._1, third._1 ), third._2 )
}

defined function intDouble
defined function DoubleInt
defined function double3

In [35]:
val myRNG = SimpleRNG( 12 )
val retVal  = intDouble(myRNG)
val retVal2 = DoubleInt( retVal._2 )
val retVal3 = double3( retVal2._2 )

myRNG: SimpleRNG = SimpleRNG(12L)
retVal: ((Int, Double), RNG) = ((4616986, -5.432967212587166E-300), SimpleRNG(217467224744870))
retVal2: ((Double, Int), RNG) = ((1.6958809269934138E-301, 1874720295), SimpleRNG(158613307474384))
retVal3: ((Double, Double, Double), RNG) = (
  (
    -1.1443258824913718E-299,
    8.723111000366053E-300,
    -7.049507585158914E-300
  ),
  SimpleRNG(198422177849677)
)

### 연습문제 6.4
정수 난수들의 목록을 생성하는 함수 작성

In [58]:
def ints(count : Int )( rng : RNG  ) : (List[Int], RNG) = {
     
    @annotation.tailrec
    def go(n : Int, list : List[Int], rand : RNG ) : (List[Int], RNG) = {
        if( n <= 0 ) (list, rand )
        else {
            val ( x,  newRng ) = rand.nextInt
            go( n -1 ,  x::list, newRng  )
        }
    }

    go( count, List[Int](), rng )
}

defined function ints

In [43]:
val myRNG = SimpleRNG( 13 )
val retVal  = ints(10)(myRNG)

myRNG: SimpleRNG = SimpleRNG(13L)
retVal: (List[Int], RNG) = (
  List(
    1609618173,
    -1228308347,
    1902795139,
    -332185025,
    -1555087323,
    -211588261,
    -1020428027,
    -697700902,
    -2132165362,
    5001735
  ),
  SimpleRNG(105487936612151)
)

(Int, RNG) 와 같이 결과값을 2개를 받아서 처리해야 하고, RNG을 기억해서 다음 인자로 넘져주어야 하는 번거로움이 있음.

=> 이런식으로 프로그램을 만들어 놓으면 욕먹음.  

=> 욕 안 먹는 순수함수 작성을 해보자. ~~~

## 6.4 상태동작을 위한 더 나은 API

In [126]:
type Rand[+A] = RNG => ( A, RNG )

defined type Rand

In [45]:
# 이거 실행시키면 아래 nonNegativeEven 오류 발생함.
val int : Rand[Int] = _.nextInt

int: RNG => (Int, RNG) = <function1>

In [127]:
def unit[A] ( a : A ) : Rand[A] = {
    rng => ( a, rng )
}

defined function unit

In [128]:
def map[A, B] (s : Rand[A])( f: A => B ) : Rand[B] = 
    rng => {
        val ( a, rng2)  = s ( rng )
        ( f(a) , rng2 )
    }

defined function map

In [79]:
def nonNegativeEven : Rand[ Int ] = {
  map( nonNegativeInt ) ( i => i - i % 2  )
}

val myRNG = SimpleRNG( 11 )
val a = nonNegativeEven( myRNG ) 
val b = nonNegativeEven( a._2 ) 

defined function nonNegativeEven
myRNG: SimpleRNG = SimpleRNG(11L)
a: (Int, RNG) = (4232236, SimpleRNG(277363943098))
b: (Int, RNG) = (178803790, SimpleRNG(11718085204285))

### 연습문제 6.5   
6.2의 double을 map을 이용해서 좀 더 우아한 방식을 구현

In [82]:
def doubleElegant : Rand[ Double ] = {
  map( nonNegativeInt ) ( i => i / (Int.MaxValue.toDouble + 1)  ) 
}

val myRNG = SimpleRNG( 12 )
val a = doubleElegant( myRNG ) 
val b = doubleElegant( a._2 ) 

defined function doubleElegant
myRNG: SimpleRNG = SimpleRNG(12L)
a: (Double, RNG) = (0.00214995164424181, SimpleRNG(302578847015))
b: (Double, RNG) = (0.4548024320974946, SimpleRNG(217467224744870))

### 연습문제 6.6
map2를 구현

In [83]:
def map2[ A, B, C ] ( ra : Rand[A], rb: Rand[B] ) ( f : (A, B) => C ) : Rand[ C ] = {
    rng => {
        val ( a,  newRa ) = ra ( rng )
        val ( b,  newRb ) = rb ( newRa )
        ( f(a, b),  newRb )
    }
}

defined function map2

<b>map2을 이용하여 임의의 RNG상태 동작을 조합할 수 있는 함수 작성 </b>

In [84]:
def both[ A, B ] ( ra : Rand[A], rb : Rand[B] ) : Rand[(A,B)] = {
    map2( ra, rb) ( (_, _) )
}

defined function both

<b>both을 이용하여 intDouble과 doubleInt을 구현 </b>

In [91]:
val rngA = SimpleRNG( 12 )
val rngB = SimpleRNG( 13 )

val randIntDouble: Rand[ (Int, Double) ] = 
    both( rngA, rngB )


: 

In [92]:
val randIntDouble: Rand[ (Int, Double) ] = 
    both( int, double )

: 

In [93]:
val randDoubleInt: Rand[ (Double, Int) ] = 
    both( double, int )

: 

### 연습문제 6.7
상태 전이들의 List를 하나의 상태전이로 조합하는 함수 sequcence를 구현하라.
구현한 sequcence를 이용하여 ints 함수를 재작성하라.

Hint : You need to recursively iterate over the list. 
Remember that you can use `foldLeft` or `foldRight` instead of writing a recursive definition. You can also reuse the `map2` function you just wrote. As a test case for your implementation, we should expect `sequence(List(unit(1), unit(2), unit(3)))(r)._1` to return `List(1, 2, 3)`


In [106]:
def sequence[ A ] ( fs : List[ Rand[A] ] ) : Rand[ List[A] ] = {
    
    val list = unit(List[A]())
    fs.foldRight( list )( (elm, listObj ) =>  map2( elm, listObj ) ( _ :: _ )  )
    
}

defined function sequence

In [143]:
def sequence[A](fs: List[Rand[A]]): Rand[List[A]] =
  fs.foldRight(unit(List[A]()))((f, acc) => map2(f, acc)(_ :: _))

defined function sequence

In [144]:
def _ints(count: Int): Rand[List[Int]] = {
    val fs = List.fill(count)( unit(1) )
    sequence( fs )
}

defined function _ints

In [145]:
val myRNG = SimpleRNG( 13 )
val retVal  = _ints(10)(myRNG)

myRNG: SimpleRNG = SimpleRNG(13L)
retVal: (List[Int], RNG) = (
  List(
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1
  ),
  SimpleRNG(13)
)

map 과 map2을 이용하면 좀더 간결하고 우아한 구현이 가능하지만, 
map 과 map2을 이용하여 구현이 어려운 경우도 있음.

예) 0 이상,  n 미만의 정수 난수를 n으로 나눈 나머지를 돌려주는 함수

In [146]:
def nonNegativeLessThan(n : Int ) : Rand[Int] = 
    map( nonNegativeInt ) { _ % n } 

val myRNG = SimpleRNG( 14 )
val retVal  = nonNegativeLessThan(50)(myRNG)

defined function nonNegativeLessThan
myRNG: SimpleRNG = SimpleRNG(14L)
retVal: (Int, RNG) = (34, SimpleRNG(353008654849))

In [ ]:
def nonNegativeLessThan(n : Int ) : Rand[Int] = {
    map( nonNegativeInt ) { i =>
        val mod = i % n
        if( i + (n-1) - mod >= 0 ) mod  
        else nonNegativeLessThan(n) (  ???  )  // nrg 객체를 넘겨줄 방법이 없음.
        
    } 
}

In [147]:
def nonNegativeLessThan(n : Int ) : Rand[Int] = { rng =>
    val (i, rng2) = nonNegativeInt( rng )
    val mod = i % n
    if( i + (n-1) - mod >= 0 ) ( mod  , rng2 )
    else nonNegativeLessThan(n) (  rng2  )  
}

defined function nonNegativeLessThan

In [151]:
val myRNG = SimpleRNG( 111 )
val retVal  = nonNegativeLessThan(50)(myRNG)
val retVal2  = nonNegativeLessThan(50)(retVal._2)

myRNG: SimpleRNG = SimpleRNG(111L)
retVal: (Int, RNG) = (27, SimpleRNG(2798854334798))
retVal2: (Int, RNG) = (36, SimpleRNG(38958739384897))

### 연습문제 6.8
flatMap을 구현하고 이것을 이용해서 nonNegativeLessThan을 구현하라.

In [156]:
// def map[A, B] (s : Rand[A])( f: A => B ) : Rand[B] = {}
def flatMap[A, B] ( f: Rand[A] ) (g:A => Rand[B] ) : Rand[B] = { 
    rng => {
       val (a, rng2) = f( rng )
       g( a )(rng2)
    }
}

defined function flatMap

In [160]:
def nonNegativeLessThan(n : Int ) : Rand[Int] = {
    flatMap( nonNegativeInt ) { i => 
       val mod = i % n
       if( i + (n-1) - mod >= 0 ) unit(mod)
       else nonNegativeLessThan(n) 
    }
}

defined function nonNegativeLessThan

In [161]:
val myRNG = SimpleRNG( 111 )
val retVal  = nonNegativeLessThan(50)(myRNG)
val retVal2  = nonNegativeLessThan(50)(retVal._2)

myRNG: SimpleRNG = SimpleRNG(111L)
retVal: (Int, RNG) = (27, SimpleRNG(2798854334798))
retVal2: (Int, RNG) = (36, SimpleRNG(38958739384897))

### 연습문제 6.9
map과 map2를 flatMap을 이용해서 다시 구현하라.

In [168]:
/*
def map[A, B] (s : Rand[A])( f: A => B ) : Rand[B] = 
    rng => {
        val ( a, rng2)  = s ( rng )
        ( f(a) , rng2 )
    }
*/
def _map[A, B] (s : Rand[A])( f: A => B ) : Rand[B] =  {
    flatMap( s ) { a => unit( f(a) ) }
}


defined function _map

In [ ]:
/*
def map2[ A, B, C ] ( ra : Rand[A], rb: Rand[B] ) ( f : (A, B) => C ) : Rand[ C ] = {
    rng => {
        val ( a,  newRa ) = ra ( rng )
        val ( b,  newRb ) = rb ( newRa )
        ( f(a, b),  newRb )
    }
}
*/
def _map2[ A, B, C ] ( ra : Rand[A], rb: Rand[B] ) ( f : (A, B) => C ) : Rand[ C ] = {
    flatMap( ra ) { a =>  
        flatMap( rb ) {  b => 
            unit( f(a, b ) )
        }
    }
}


// 정답
def _map2[A,B,C](ra: Rand[A], rb: Rand[B])(f: (A, B) => C): Rand[C] =
  flatMap(ra)(a => map(rb)(b => f(a, b)))

하나 모자라는 오류를 있는 주사위 굴림 함수 

In [169]:
def rollDie : Rand[Int] = nonNegativeLessThan(6)
val zero = rollDie( SimpleRNG(5))._1

defined function rollDie
zero: Int = 0

버그가 간단히 교정된 주사위 굴림 함수 

In [170]:
def rollDie : Rand[Int] = map( nonNegativeLessThan(6) ) ( _ + 1)
val zero = rollDie( SimpleRNG(5))._1

defined function rollDie
zero: Int = 1

## 6.5 일반적 상태 동작 자료 형식

map을 RNG 상태 동작에 상관하지 않고, 일반적인 서명을 부여함.

In [180]:
// def map[A, B] (s : Rand[A])( f: A => B ) : Rand[B] 
def map[ S, A, B ] ( a : S => ( A, S ) ) (f : A => B ) : S  => ( B, S )

: 

임의의 상태를 처리하는 Rand보다 더 일반적인 형식
<font color="red">State</font>는 <b>어떤 상태를 유지하는 계산</b>, 즉 <b>상태 동작</b> 또는 <b>상태 전이</b>를 대표하고, 명령문을 대표한다.

In [181]:
// type Rand[+A] = RNG => ( A, RNG )
type State[S, +A ] = S => (A ,S)

defined type State

In [182]:
case class State[S, +A] ( run : S => (A, S))

defined class State

Rand을 State를 이용해서 형식별칭 정의

In [172]:
type Rand[A] = State[RNG, A]

defined type Rand

### 연습문제 6.10
함수 unit, map, map2, flatMap, sequence를 일반화하라.

In [189]:
/*
def unit[A] ( a : A ) : Rand[A] = 
    rng => ( a, rng )
*/
def unit[S, A]( a : A ) : State[S, A] = {
    State ( s => ( a, s ) )
}

defined function unit

## 6.6 순수함수적 명령식 프로그래밍

In [173]:
val ns : Rand[ List[Int] ] = 
   int.flatMap( x => 
        int.flatMap( y => 
            ints(x).map( xs => 
                xs.map( _ % y ) 
            )
        )
   )

: 

In [174]:
val ns : Rand[ List[Int] ] = for {
    x <- int
    y <- int
    xs <- ints(x)
} yield xs.map ( _ % y )

: 

In [178]:
def modify[S] ( f: S => S ) : State[S, Unit] = for {
    s <- get
    _ <- set( f(s) ) 
} yield {}

: 

In [177]:
def get[S] : State[S, S] = State( s => (s, s) )

defined function get

In [176]:
def set[S]( s :S ) : State[S, Unit] = State( _ => ( (), s)  )

defined function set

### 연습문제 6.11
사탕 판매가를 본뜬 유한상태 머신을 구현하라.